In [ ]:
## gait cycle analysis

## 歩行足圧中心軌跡 -> 比べるには向かない
## line chart -> 比べるのに向いている ????
## threasholdどうしよう？
## 平均でいいのでは？

## stance phase: initial contact -> flat foot -> heel off -> toe off
## swing  phase:

## stance phase: initial contact -> (other toe off) -> flat foot -> heel off -> (other initial contact) -> toe off
## swing  phase:

## from pressure
### cadence: (steps/min)
### step duaration:  (sec)
### stance duaration:(sec)
### swing duaration: (sec)
### both feet duaration:(sec)

### each sensor compare to mean 
### each region compare to mean
#### toe  #14~16    
#### mid-toe #9~13
#### mid-heel #5~8
#### heel #1~4

#### toe #9~11,14~16
#### middle #5~8, 12,13
#### heel #1~4

## from imu
### walking length (m)
### walking speed (m/s)
### acceleration  (m/ss)

### txt outline ###
#   0  |     1~16          |    17~19            |  20 ~ 22         | 23               |  24,25
# time | left sensors 1~16 | left accelerate xyz | left angular xyz | left total force | left CoP xy | 

#      |     26~41         |    42~44            |  45 ~ 47         | 48               |  49,50
#      | left sensors 1~16 | left accelerate xyz | left angular xyz | left total force | left CoP xy | 
###

import matplotlib.pyplot as plt
import numpy as np

def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    header_line = None
    for i, line in enumerate(lines):
        if line.startswith('# time'):
            header_line = i
            break

    if header_line is None:
        raise ValueError("Header line not found in the file")

    columns = lines[header_line].strip().split('\t')
    left_pressure_indices = [i for i, col in enumerate(columns) if col.startswith('left pressure')]
    right_pressure_indices = [i for i, col in enumerate(columns) if col.startswith('right pressure')]
    ioU_indices = [i for i, col in enumerate(columns) if 'left acceleration' in col or 'left angular' in col or 'right acceleration' in col or 'right angular' in col]

    data_lines = lines[header_line + 1:]
    pressure_data = []
    ioU_data = []

    for line in data_lines:
        values = line.strip().split('\t')
        if len(values) >= max(left_pressure_indices + right_pressure_indices + ioU_indices):
            try:
                pressure_row = [float(values[j]) for j in left_pressure_indices + right_pressure_indices if values[j]]
                ioU_row = [float(values[j]) for j in ioU_indices if values[j]]
                if len(pressure_row) == 32 and len(ioU_row) == 12:
                    pressure_data.append(pressure_row)
                    ioU_data.append(ioU_row)
            except ValueError:
                continue

    return np.array(pressure_data), np.array(ioU_data)
    
def calc_pres_ragion(pressures):
#### toe  #14~16    
#### mid-toe #9~13
#### mid-heel #5~8
#### heel #1~4

#### toe #9~11,14~16
#### middle #5~8, 12,13
#### heel #1~4
    pres_len = len(pressures)
    l_toe_pres = [sum(pressures[i][13:16]) for i in range(0, pres_len)]
    l_mid_toe_pres  = [sum(pressures[i][8:13]) for i in range(0, pres_len)]
    l_mid_heel_pres = [sum(pressures[i][4:8]) for i in range(0, pres_len)]
    l_heel_pres = [sum(pressures[i][0:4]) for i in range(0, pres_len)]

    r_toe_pres = [sum(pressures[i][29:32]) for i in range(0, pres_len)]
    r_mid_toe_pres  = [sum(pressures[i][24:29]) for i in range(0, pres_len)]
    r_mid_heel_pres = [sum(pressures[i][20:24]) for i in range(0, pres_len)]
    r_heel_pres = [sum(pressures[i][16:20]) for i in range(0, pres_len)]

    pres = np.array([l_toe_pres, l_mid_toe_pres, l_mid_heel_pres, l_heel_pres, r_toe_pres, r_mid_toe_pres, r_mid_heel_pres, r_heel_pres])

    return pres

def calc_stances(pres):
#    pres = [l_toe_pres, l_mid_toe_pres, l_mid_heel_pres, l_heel_pres, r_toe_pres, r_mid_toe_pres, r_mid_heel_pres, r_heel_pres]

    # calc stances
    ## stance phase: initial contact -> flat foot -> heel off -> toe off
    ## mid stance: CoF is in the  middle_toe ... some do not have this phase

    l_init_heel = on_frames(pres[3])
    l_flat_foot = on_frames(pres[1])
    l_off_heel  = off_frames(pres[3])
    l_off_toe   = off_frames(pres[0])

    r_init_heel = on_frames(pres[7])
    r_flat_foot = on_frames(pres[5])
    r_off_heel  = off_frames(pres[7])
    r_off_toe   = off_frames(pres[4])
    
    return [l_init_heel, l_flat_foot, l_off_heel, l_off_toe, r_init_heel, r_flat_foot, r_off_heel, r_off_toe]

def on_frames(pres):
    # 3 frame more than average -> start of something
    rst = []
    time = np.arange(0, len(pres)*0.02, 0.02)
    
    if pres.ndim > 1:
        pres = np.array([sum(l) for l in pres])
    avr = np.average(pres)

    for i in range(len(pres)-2):
        if pres[i] > avr and pres[i+1] > avr and pres[i+2] > avr:
            rst.append(time[i])

    return rst

def off_frames(pres):
    # 3 frame more than average -> start of something
    rst = []
    time = np.arange(0, len(pres)*0.02, 0.02)

    if pres.ndim > 1:
        pres = np.array([sum(l) for l in pres])
    avr = np.average(pres)

    for i in range(2,len(pres)):
        if pres[i-2] < avr and pres[i-1] < avr and pres[i] < avr:
            rst.append(time[i])

    return rst

def plot_pressure(pressures, stance):
    # pres  = [r_toe_pres, l_mid_toe_pres, l_mid_heel_pres, l_heel_pres, r_toe_pres, r_mid_toe_pres, r_mid_heel_pres, r_heel_pres]
    # stance= [l_init_heel, l_flat_foot, l_off_heel, l_off_toe, r_init_heel, r_flat_foot, r_off_heel, r_off_toe]

    pres_len = len(pressures[0])
    time = np.arange(0, pres_len*0.02, 0.02)
    y_max = np.max(pressures) + 5 # +5は独断と偏見

    # plot pressure
    fig,axs = plt.subplots(2,1)
    axs[0].plot(time, pressures[0], label='toe')
    axs[0].plot(time, pressures[1], label='mid_toe')
    axs[0].plot(time, pressures[2], label='mid_heel')
    axs[0].plot(time, pressures[3], label='heel')
    axs[0].set_title("left foot pressures")
    axs[1].plot(time, pressures[4], label='toe')
    axs[1].plot(time, pressures[5], label='mid_toe')
    axs[1].plot(time, pressures[6], label='mid_heel')
    axs[1].plot(time, pressures[7], label='heel')
    axs[1].set_title("right foot pressures")

    # plot stances
    # axs[0].vlines(stance[0], ymin=0, ymax=y_max, color='b', label='heel on')
    # axs[0].vlines(stance[1], ymin=0, ymax=y_max, color='r', label='flat foot')
    # axs[0].vlines(stance[2], ymin=0, ymax=y_max, color='g', label='heel off')
    # axs[0].vlines(stance[3], ymin=0, ymax=y_max, color='y', label='toe of')
    # axs[1].vlines(stance[4], ymin=0, ymax=y_max, color='b', label='heel on')
    # axs[1].vlines(stance[5], ymin=0, ymax=y_max, color='r', label='flat foot')
    # axs[1].vlines(stance[6], ymin=0, ymax=y_max, color='g', label='heel off')
    # axs[1].vlines(stance[7], ymin=0, ymax=y_max, color='y', label='toe of')

    fig.tight_layout()
    plt.show()

In [ ]:
# summarize stance length
# bar chart comparison over poses
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import csv
import glob
import os

# pose_labels = ['S1', 'S2', 'S3', 'S4']
def get_class_label(file_path):
    if "weight_good" in file_path:
        return "S1"
    elif "weight_high" in file_path:
        return "S1"
    elif "weight_hand" in file_path:
        return "S2"
    elif "weight_torso" in file_path:
        return "S3"
    elif "weight_bad" in file_path:
        return "S4"
    
def get_participant_label(file_path):
    if "tabei" in file_path:
        return "P1"
    elif "0917/ito" in file_path:
        return "P2"
    elif "hirano" in file_path:
        return "P3"
    elif "peng" in file_path:
        return "P4"
    elif "1031" in file_path:
        return "P2_day2"
    
# sync with csv
time_label = ['init_heel', 'flat_foot', 'off_heel', 'off_toe']

bar1_folder = ['ito_data/0917/tabei','ito_data/0917/ito','ito_data/0916/hirano','ito_data/1004/peng','ito_data/0828/peng']
bar2_folder = ['ito_data/1031']

# hue_label = ['P1', 'P2', 'P3', 'P4', 'P2_day2']
# pose_labels = ['S1', 'S2', 'S3', 'S4']

fig_path = 'plot/1024/weight/summary/'

## csv header 
#  (["L/R", "label", "time", "pressure", "time_percentage"])
## df header
# list_columns = ['file_path', 'participant', 'pose', "L/R", "label", "time", "pressure", "time_percentage"]

df_lst = []
for j in range(len(bar1_folder)):
    files = glob.glob(os.path.join(bar1_folder[j], "*.csv"))
    for file in files:
        df_tmp = pd.read_csv(file)

        # add labels
        df_tmp['file_path']=file
        df_tmp['participant']=get_participant_label(file)
        df_tmp['pose']=get_class_label(file)
        df_lst.append(df_tmp)
    
for j in range(len(bar2_folder)):
    files = glob.glob(os.path.join(bar2_folder[j], "*.csv"))
    for file in files:
        df_tmp = pd.read_csv(file)

        # add labels
        df_tmp['file_path']=file
        df_tmp['participant']=get_participant_label(file)
        df_tmp['pose']=get_class_label(file)
        df_lst.append(df_tmp)

df = pd.concat(df_lst, axis=0)

# df_stance = df[df['label'] == "stance_duration"]
# df_stance = df[df['label'].isin(["stance_duration"])]
timing = time_label[1:len(time_label)-1]
df_stance = df.query('label in ["stance_duration"]')
df_timing = df.query(f'label in {timing}')
df_pres = df.query('~pressure.isnull()')

# stance length summary
summary_df = df_stance.groupby(['L/R','label','participant', 'pose']).agg(
    mean_time=('time', 'mean'),
    stddev_time=('time', 'std')
).reset_index()
output_csv_file = fig_path+"stance_duaration.csv"
summary_df.to_csv(output_csv_file, index=False)

# stance timing summary
summary_df = df_timing.groupby(['L/R','label','participant', 'pose']).agg(
    mean_time_perc=('time_percentage', 'mean'),
    stddev_time_perc=('time_percentage', 'std')
).reset_index()
output_csv_file = fig_path+"stance_timing.csv"
summary_df.to_csv(output_csv_file, index=False)

# force summary
summary_df = df_pres.groupby(['L/R','label','participant', 'pose']).agg(
    mean_pres=('pressure', 'mean'),
    stddev_pres=('pressure', 'std'),
    mean_time_perc=('time_percentage', 'mean'),
    stddev_time_perc=('time_percentage', 'std')
).reset_index()
output_csv_file = fig_path+"force_timing.csv"
summary_df.to_csv(output_csv_file, index=False)

df_timing["L/R_label"] = df_timing["L/R"]+df_timing["label"]
df_pres["L/R_label"]   = df_pres["L/R"]  +df_pres["label"]

plt.rcParams["font.size"] = 5
plot = sns.catplot(x='pose', y='time', hue='participant', data=df_stance, errorbar="sd", kind="box")
plot.figure.suptitle('stance event timing')
plt.grid()
plot.savefig(fig_path+"stance_length.png")

plt.rcParams["font.size"] = 5
g = sns.FacetGrid(df_timing, row="L/R_label", col='participant', margin_titles=True)
g.map(sns.boxplot, "pose", "time_percentage")
plot.savefig(fig_path+"stance_timing.png")

plt.rcParams["font.size"] = 5
g = sns.FacetGrid(df_pres, row="L/R_label", col='participant', margin_titles=True)
g.map(sns.boxplot, "pose", "time_percentage")
plot.savefig(fig_path+"max_force_timing.png")

plt.rcParams["font.size"] = 5
g = sns.FacetGrid(df_pres, row="L/R_label", col='participant', margin_titles=True)
g.map(sns.boxplot, "pose", "pressure")
plot.savefig(fig_path+"max_force.png")

In [ ]:
file_path = 'stg'

pressures, IoU = parse_file(file_path)
pressures = calc_pres_ragion(pressures)
stances  = calc_stances(pressures)
plot_pressure(pressures, stances)

In [21]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from PIL import Image, ImageDraw
import numpy as np
import io

def plot_insole_heatmap_gif(insole_image, sensor_values, output_path):

    # Adjusted sensor positions based on the image dimensions (296x666)
    sensor_positions = [
        (159, 598), (95, 604), (163, 512), (89, 515),
        (167, 419), (89, 428), (212, 308), (62, 334),
        (240, 182), (187, 183), (145, 187), (105, 197),
        (61, 207), (223, 71), (167, 79), (102, 92)
    ]
    # Create a composite image
    width, height = insole_image.size
    
    frames = []
    #left foot
    for frame in range(len(sensor_values)):
        composite_image = Image.new('RGBA', (width * 2, height))
        img = insole_image.copy()
        
        # Draw left foot sensors
        draw = ImageDraw.Draw(img, "RGBA")
        for i, (x, y) in enumerate(sensor_positions):
            radius = 20
            color = plt.cm.viridis(sensor_values[frame, i+16])
            color = tuple(int(255 * c) for c in color[:3]) + (int(255 * 0.8),)  # Add alpha
            draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill=color)
        composite_image.paste(img, (0, 0))

        # Draw right foot sensors
        draw = ImageDraw.Draw(img, "RGBA")
        for i, (x, y) in enumerate(sensor_positions):
            radius = 20
            color = plt.cm.viridis(sensor_values[frame, i ])
            color = tuple(int(255 * c) for c in color[:3]) + (int(255 * 0.8),)  # Add alpha
            draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill=color)
        composite_image.paste(img.transpose(Image.FLIP_LEFT_RIGHT), (width, 0))

        frames.append(composite_image)
    
    # Save frames as a gif
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=100)


In [24]:
import pandas as pd

# Load the insole background
insole_image = Image.open('insole.png')
# Specify the output gif path
output_gif_path = 'insole_animation.gif'

leftp_idx = [i for i in range(1,17)]
rightp_idx = [i for i in range(26,42)]
df = pd.read_csv('data/insole/day1/shiori-ratio1.csv', header=None)

data = pd.concat([df.iloc[:, leftp_idx], df.iloc[:, rightp_idx]], axis=1)

# Plot the heatmap and save as gif
plot_insole_heatmap_gif(insole_image, data.values, output_gif_path)

In [ ]:
from PIL import Image
import csv
import numpy as np
import matplotlib.pyplot as plt

def plot_3d_pose(pose, ax):
    ax.clear()
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 5)
    ax.set_zlim(0, 5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    # for joint in pose:
        # ax.scatter(joint[0], joint[1], joint[2], c='b', marker='o')
    
    # create bones
    left_bone  = np.array([pose[0], pose[2], pose[4], pose[6], pose[8], pose[15], pose[16]]).T  # foot_tip, ankle, knee, hip, chest, neck, head
    right_bone = np.array([pose[1], pose[3], pose[5], pose[7], pose[8], pose[15], pose[16]]).T 
    left_hand  = np.array([pose[8], pose[ 9], pose[11], pose[13]]).T  # chest, shoulder, elbow, wrist
    right_hand = np.array([pose[8], pose[10], pose[12], pose[14]]).T 

    ax.plot(left_bone[0],  left_bone[1],  left_bone[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_bone[0], right_bone[1], right_bone[2], c='b',  marker='.', linestyle='solid')
    ax.plot(left_hand[0],  left_hand[1],  left_hand[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_hand[0], right_hand[1], right_hand[2], c='b', marker='.', linestyle='solid')

    ax.view_init(elev=30, azim=45)  # Set a good view angle

def parse_csv_bvh_17(file_path):
    jointNameList = [
        'pelvis',
        'abdomen',
        'thorax',
        'l_clavicle', 'l_uarm', 'l_larm', 'l_hand', 'l_lathand', 'end:l_hand', 'l_medhand', 'end:thorax:l',
        'r_clavicle', 'r_uarm', 'r_larm', 'r_hand', 'r_lathand', 'end:r_hand', 'r_medhand', 'end:thorax:r',
        'neck',
        'head',
        'r_ear', 'end:head', 'l_ear', 'end:head', 'l_eye', 'end:head', 'r_eye', 'end:head',
        'nose','end:pelvis',
        'l_thigh',
        'l_shank',
        'l_foot',
        'l_toes', 'l_toe', 'end:l_toes', 'l_f_b_toe', 'end:l_toes', 'l_f_m_toe','end:pelvis', # b_toe -> thumb, m_toe -> index
        'r_thigh',
        'r_shank',
        'r_foot',
        'r_toes', 'r_toe', 'end:r_toes', 'r_f_b_toe', 'end:r_toes', 'r_f_m_toe','end:'
    ]

    # # Joint labels
    # joint_labels = [
    #     "foot_tip_L", "foot_tip_R", "ankle_L", "ankle_R",
    #     "knee_L", "knee_R", "hip_L", "hip_R",
    #     "chest", "shoulder_L", "shoulder_R", "elbow_L",
    #     "elbow_R", "wrist_L", "wrist_R", "neck", "head"
    # ]

    joint17_labels = [
        "l_toe",   "r_toe",   "l_foot",  "r_foot",
        "l_shank", "r_shank", "l_thigh", "r_thigh",
        "thorax",  "l_uarm",  "r_uarm",  "l_larm",
        "r_larm",  "l_hand",  "r_hand",  "neck",   "head"
    ]

    poses = []
    pose  = [[0]*3]*len(joint17_labels)
    print(len(pose))
    time  = 0
    i = 0
    # create correct sets of joints
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            # row = timestamp,jointName,position_x,position_y,position_z
            if i == 0: # init params
                time = row[0]
                pose  = [[0]*3]*len(joint17_labels)
                if float(time) >10:
                    break

            # check labels, add joint lists
            if row[0] != time:
                print('time('+str(time)+') interupted at joint: '+jointNameList[i])
            elif row[1] == jointNameList[i]: # correct
                i += 1
                if row[1] in joint17_labels:
                    pose[joint17_labels.index(row[1])] = [float(row[2]), float(row[3]), float(row[4])]
            else: # time correct, incorrect joint
                raise ValueError("incorrect joint detected")
            
            # all joint found -> append to the pose list
            if i == len(jointNameList):
                poses.append(pose)
                i = 0

    return np.array(poses)

def parse_csv_bvh(file_path):
    jointNameList = [
        'pelvis',
        'abdomen',
        'thorax',
        'l_clavicle', 'l_uarm', 'l_larm', 'l_hand', 'l_lathand', 'end:l_hand', 'l_medhand', 'end:thorax:l',
        'r_clavicle', 'r_uarm', 'r_larm', 'r_hand', 'r_lathand', 'end:r_hand', 'r_medhand', 'end:thorax:r',
        'neck',
        'head',
        'r_ear', 'end:head', 'l_ear', 'end:head', 'l_eye', 'end:head', 'r_eye', 'end:head',
        'nose','end:pelvis',
        'l_thigh',
        'l_shank',
        'l_foot',
        'l_toes', 'l_toe', 'end:l_toes', 'l_f_b_toe', 'end:l_toes', 'l_f_m_toe','end:pelvis',
        'r_thigh',
        'r_shank',
        'r_foot',
        'r_toes', 'r_toe', 'end:r_toes', 'r_f_b_toe', 'end:r_toes', 'r_f_m_toe','end:'
    ]

    poses = []
    pose  = []
    time  = 0
    i = 0
    # create correct sets of joints
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            # row = timestamp,jointName,position_x,position_y,position_z
            if i == 0: # init params
                time = row[0]
                pose = []
                if float(time) >10:
                    break

            # check labels, add joint lists
            if row[0] != time:
                print('time('+str(time)+') interupted at joint: '+jointNameList[i])
            elif row[1] == jointNameList[i]: # correct
                pose.append([float(row[2]),float(row[3]),float(row[4])])
                i += 1
            else: # time correct, incorrect joint
                raise ValueError("incorrect joint detected")
            
            # all joint found -> append to the pose list
            if i == len(jointNameList):
                poses.append(pose)
                i = 0

    return np.array(poses)


# Define a function to create a 3D plot of the pose
def plot_3d_bvh_joints(pose, ax):
    ax.clear()
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 5)
    ax.set_zlim(0, 5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    # for joint in pose:
        # ax.scatter(joint[0], joint[1], joint[2], c='b', marker='o')
    
    # create bones
    left_bone  = np.array([pose[30], pose[29], pose[20], pose[19],pose[2], pose[1], pose[0], pose[31], pose[32], pose[33], pose[34], pose[35], pose[36]]).T # end, nose, head, neck, thoarx, abdomen, plevis, thigh, shank, foot, toes, toe, end
    right_bone = np.array([pose[30], pose[29], pose[20], pose[19],pose[2], pose[1], pose[0], pose[41], pose[42], pose[43], pose[44], pose[45], pose[46]]).T 
    left_eye   = np.array([pose[20], pose[25], pose[26]]).T  # head, eye, end
    right_eye  = np.array([pose[20], pose[27], pose[28]]).T 
    left_ear   = np.array([pose[20], pose[23], pose[24]]).T  # head, ear, end
    right_ear  = np.array([pose[20], pose[21], pose[22]]).T 
    left_btoe  = np.array([pose[34], pose[37], pose[38]]).T  # toes, f_b_toe, end
    right_btoe = np.array([pose[44], pose[47], pose[48]]).T  
    left_mtoe  = np.array([pose[34], pose[39], pose[40]]).T  # toes, f_m_toe, end
    right_mtoe = np.array([pose[44], pose[49], pose[50]]).T  
    left_hand  = np.array([pose[ 2], pose[ 3], pose[ 4], pose[ 5], pose[ 6], pose[ 7], pose[10]]).T  # thorax, clavicle, uarm, larm, hand, medhand, end
    right_hand = np.array([pose[ 2], pose[11], pose[12], pose[13], pose[14], pose[15], pose[18]]).T 
    left_lhand = np.array([pose[ 6], pose[ 8], pose[ 9]]).T  # hand, lathend, end
    right_lhand= np.array([pose[14], pose[16], pose[17]]).T 

    ax.plot(left_bone[0],  left_bone[1],  left_bone[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_bone[0], right_bone[1], right_bone[2], c='b', marker='.', linestyle='solid')
    ax.plot(left_eye[0],   left_eye[1],   left_eye[2],   c='b', marker='.', linestyle='solid')
    ax.plot(right_eye[0],  right_eye[1],  right_eye[2],  c='b', marker='.', linestyle='solid')
    ax.plot(left_ear[0],   left_ear[1],   left_ear[2],   c='b', marker='.', linestyle='solid')
    ax.plot(right_ear[0],  right_ear[1],  right_ear[2],  c='b', marker='.', linestyle='solid')
    ax.plot(left_btoe[0],  left_btoe[1],  left_btoe[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_btoe[0], right_btoe[1], right_btoe[2], c='b', marker='.', linestyle='solid')
    ax.plot(left_mtoe[0],  left_mtoe[1],  left_mtoe[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_mtoe[0], right_mtoe[1], right_mtoe[2], c='b', marker='.', linestyle='solid')
    ax.plot(left_hand[0],  left_hand[1],  left_hand[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_hand[0], right_hand[1], right_hand[2], c='b', marker='.', linestyle='solid')
    ax.plot(left_hand[0],  left_hand[1],  left_hand[2],  c='b', marker='.', linestyle='solid')
    ax.plot(right_hand[0], right_hand[1], right_hand[2], c='b', marker='.', linestyle='solid')
    ax.plot(left_lhand[0], left_lhand[1], left_lhand[2], c='b', marker='.', linestyle='solid')
    ax.plot(right_lhand[0],right_lhand[1],right_lhand[2],c='b', marker='.', linestyle='solid')

    ax.view_init(elev=30, azim=45)  # Set a good view angle

# Create frames for the GIF
def create_gif(pose_data, output_path, bvh_all=False):
    frames = []
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for t in range(pose_data.shape[0]):
        if bvh_all:
            plot_3d_bvh_joints(pose_data[t], ax)
        else:
            plot_3d_pose(pose_data[t], ax)
        fig.canvas.draw()
        
        # Convert the plot to a PIL image
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        img = Image.fromarray(img)
        
        frames.append(img)
    
    # Save frames as a GIF
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=100)
    plt.close(fig)

In [20]:
import csv

# Specify the output gif path
input_file_path = 'data/qualisys/motionList_yoga3.csv'
output_file_path = 'data/qualisys/joints17List_yoga3.csv'

pose = parse_csv_bvh_17(input_file_path)
# create_gif(pose, output_gif_path, bvh_all=False)
with open(output_file_path, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(pose)

17


In [ ]:
from IPython.display import Image as IPImage

# Display the gif in Jupyter Notebook
IPImage(filename=output_gif_path)